# Memory Exercises - Solutions

## Memory in LangGraph - Quick Checks

### Solutions to Quick Check Questions

1. **Understanding Persistence**
   - A checkpointer like `InMemorySaver` stores checkpoints of the workflow state. Without a checkpointer, each invocation starts from scratch; with it, the state is saved and can be resumed across runs.

2. **State vs. Config**
   - The **state** contains the data that flows through the nodes (e.g., messages, counters, user memory). The **config** provides runtime parameters that influence how the workflow runs (e.g., thread IDs, language settings) without altering the state.

3. **Thread Isolation**
   - Each thread is identified by a unique `thread_id` in the config. LangGraph uses this identifier to keep the state of each conversation separate, ensuring that data doesn't leak between users.

## Coding Exercise: Implementing Persistent Memory - Solution

Below is one way to implement the memory features. Solutions may differ.

This solution uses `InMemorySaver` to persist the user's preferences between invocations, track visits, and personalize the assistant's replies.

In [1]:
from typing import TypedDict, Dict, Any, List
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

class MemoryState(TypedDict):
    messages: List[str]
    user_id: str
    user_memory: Dict[str, Any]  # long-term memory for preferences and counts

# Node that updates user memory and personalizes responses
def remember_preferences(state: MemoryState) -> MemoryState:
    # Get the last message in lowercase for preference detection
    last_msg = state["messages"][-1].lower() if state["messages"] else ""

    # Initialize diet list
    diet = state["user_memory"].get("diet", [])

    # Detect dietary preferences and store them
    if "vegan" in last_msg and "vegan" not in diet:
        diet.append("vegan")
    if "vegetarian" in last_msg and "vegetarian" not in diet:
        diet.append("vegetarian")
    if "gluten-free" in last_msg or "gluten free" in last_msg:
        if "gluten-free" not in diet:
            diet.append("gluten-free")

    # Update the diet in user memory
    if diet:
        state["user_memory"]["diet"] = diet

    # Track visits
    count = state["user_memory"].get("visits", 0) + 1

     # Add a welcome-back message on return visits
    if count > 1:
        state["messages"].append(f"Welcome back! This is visit #{count}")


    state["user_memory"]["visits"] = count

    # Personalize response based on preferences
    if diet:
        # Suggest a dish based on the first preference listed
        if "vegan" in diet:
            state["messages"].append("May I recommend our vegan Buddha bowl?")
        elif "vegetarian" in diet:
            state["messages"].append("How about a fresh vegetarian salad?")
        elif "gluten-free" in diet:
            state["messages"].append("Try our gluten-free pasta.")



    return state

# Build the workflow
workflow = StateGraph(MemoryState)
workflow.add_node("remember", remember_preferences)
workflow.add_edge(START, "remember")
workflow.add_edge("remember", END)

# Compile with an in-memory checkpointer to enable persistence across runs
app = workflow.compile(checkpointer=InMemorySaver())


### Testing the Implementation

In [2]:
# config with a unique thread_id
config = {"configurable": {"thread_id": "user123"}}

# First interaction: user states a dietary preference
state1 = {
    "messages": ["I'm vegan and looking for options"],
    "user_id": "user123",
    "user_memory": {}
}

result1 = app.invoke(state1, config=config)
print("First interaction - User Memory:", result1["user_memory"])
# Expected: {'diet': ['vegan'], 'visits': 1}

print("First interaction - Messages:", result1["messages"])
# Expected to include a vegan dish suggestion


First interaction - User Memory: {'diet': ['vegan'], 'visits': 1}
First interaction - Messages: ["I'm vegan and looking for options", 'May I recommend our vegan Buddha bowl?']


In [3]:
# Second interaction resumes from saved state
saved_state = app.get_state(config).values

# User adds another preference
saved_state["messages"].append("I'm also gluten-free")
result2 = app.invoke(saved_state, config=config)

# Show the latest messages and memory
print("Second interaction - Messages (last 2):", result2["messages"][-2:])
# Expected: a gluten-free suggestion and a welcome-back message

print("Second interaction - User Memory:", result2["user_memory"])
# Expected: {'diet': ['vegan', 'gluten-free'], 'visits': 2}


Second interaction - Messages (last 2): ['Welcome back! This is visit #2', 'May I recommend our vegan Buddha bowl?']
Second interaction - User Memory: {'diet': ['vegan', 'gluten-free'], 'visits': 2}


#### Key Concepts Demonstrated:

1. **Memory Persistence**: The `InMemorySaver` checkpointer ensures that state is preserved between invocations

2. **Thread Isolation**: Each `thread_id` maintains its own separate state, allowing multiple users to interact without interference

3. **State Evolution**: The `user_memory` dictionary accumulates information over time, enabling personalized interactions
